In [1]:
import pandas as pd
import numpy as np

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# import torch.nn as nn
# from torch.utils.data import DataLoader
# import torch.optim as optim
# import torch
from sklearn.model_selection import train_test_split
import os
import sys
from sklearn.metrics import accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report


In [3]:
# Set the root directory of your repo as part of the path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [4]:
data = pd.read_csv('../Data/combined/full_lathan.csv')

/tmp/ipykernel_217845/4119657318.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/combined/full_lathan.csv')


In [5]:
from src.data.preprocess import (
    combine_team_rows,
    add_home_win_column,
    split_wins_losses,
    add_playoff_indicator,
    combine_date_columns,
    remove_all_star_and_playoff_games,
    create_ml_dataset

)

In [6]:
# Creating key predictor columns
data2 = create_ml_dataset(data)

In [7]:
# Filling in GAME_DATE_EST Nans
data2['GAME_DATE_EST'] = data2['Date']

In [8]:
data2.columns

Index(['GAME_ID', 'GAME_SEQUENCE', 'Date', 'YEAR', 'GAME_DATE_EST', 'a_index',
       'a_TEAM_ID', 'a_TEAM_WINS_LOSSES', 'a_PTS', 'a_FG_PCT', 'a_FT_PCT',
       'a_FG3_PCT', 'a_AST', 'a_REB', 'a_TOV', 'h_index', 'h_TEAM_ID',
       'h_TEAM_WINS_LOSSES', 'h_PTS', 'h_FG_PCT', 'h_FT_PCT', 'h_FG3_PCT',
       'h_AST', 'h_REB', 'h_TOV', 'HOME_WIN', 'a_WINS', 'a_LOSSES', 'h_WINS',
       'h_LOSSES', 'a_SEASON_GAMES_PLAYED', 'h_SEASON_GAMES_PLAYED',
       'IS_PLAYOFF_GAME', 'a_season_PTS_lag', 'a_season_AST_lag',
       'a_season_REB_lag', 'a_season_TOV_lag', 'a_season_FG_PCT_avg',
       'a_season_FT_PCT_avg', 'a_season_FG3_PCT_avg', 'a_WIN_PCT',
       'a_WIN_PCT_LAST10', 'h_season_PTS_lag', 'h_season_AST_lag',
       'h_season_REB_lag', 'h_season_TOV_lag', 'h_season_FG_PCT_avg',
       'h_season_FT_PCT_avg', 'h_season_FG3_PCT_avg', 'h_WIN_PCT',
       'h_WIN_PCT_LAST10'],
      dtype='object')

In [9]:
# Adding season columns (ex: Season 2012-2013 is season 1)
season_order = sorted(data2['YEAR'].unique())
season_to_index = {season: i+1 for i, season in enumerate(season_order)}
data2['SEASON_NUM'] = data2['YEAR'].map(season_to_index)

# Dropping rows with Missing values for now
data2 = data2.dropna()

In [10]:
# Filtering data into train (2012 - 2016), validation (2017), test seasons (2018-2019)
train_df = data2[data2['SEASON_NUM'] <= 3]    # Seasons 1-3
val_df = data2[data2['SEASON_NUM'] == 4]      # Season 4
test_df = data2[data2['SEASON_NUM'] >= 5]     # Seasons 5-6

In [12]:
# Dropping columns that we don't need?
columns_to_drop = ['a_PTS', 'a_FG_PCT', 'a_FT_PCT','a_FG3_PCT', 'a_AST', 'a_REB', 'a_TOV','GAME_ID',
                   'h_PTS', 'h_FG_PCT', 'h_FT_PCT', 'h_FG3_PCT','h_AST', 'h_REB', 'h_TOV',
                   'GAME_SEQUENCE', 'Date', 'YEAR', 'a_index', 'a_TEAM_ID','h_TEAM_ID', 'a_TEAM_WINS_LOSSES',
                  'h_TEAM_WINS_LOSSES', 'IS_PLAYOFF_GAME', 'GAME_DATE_EST', 'h_index','SEASON_NUM', 'HOME_WIN']


# Creating train, validation, and test sets
X_train = train_df.drop(columns_to_drop, axis = 1)
y_train = train_df['HOME_WIN']

X_val = val_df.drop(columns_to_drop, axis = 1)
y_val = val_df['HOME_WIN']

X_test = test_df.drop(columns_to_drop, axis = 1)
y_test = test_df['HOME_WIN']

In [13]:
# Checking Nans totals for differnet columns
X_train.isna().sum().sort_values(ascending=False)

a_WINS                   0
a_LOSSES                 0
h_WINS                   0
h_LOSSES                 0
a_SEASON_GAMES_PLAYED    0
h_SEASON_GAMES_PLAYED    0
a_season_PTS_lag         0
a_season_AST_lag         0
a_season_REB_lag         0
a_season_TOV_lag         0
a_season_FG_PCT_avg      0
a_season_FT_PCT_avg      0
a_season_FG3_PCT_avg     0
a_WIN_PCT                0
a_WIN_PCT_LAST10         0
h_season_PTS_lag         0
h_season_AST_lag         0
h_season_REB_lag         0
h_season_TOV_lag         0
h_season_FG_PCT_avg      0
h_season_FT_PCT_avg      0
h_season_FG3_PCT_avg     0
h_WIN_PCT                0
h_WIN_PCT_LAST10         0
dtype: int64

In [14]:
# Converting data to arrays for compatible Tabnet input
X_train = X_train.values
y_train = y_train.values

X_val = X_val.values
y_val = y_val.values

X_test = X_test.values
y_test = y_test.values

In [15]:
# Testing TabNet Model
clf = TabNetClassifier(
    n_d=16,               # Number of decision prediction steps
    n_a=16,               # Number of attention steps (usually equal to n_d)
    n_steps=5,            # Number of steps in TabNet
    gamma=1.5,            # Relaxation factor
    lambda_sparse=1e-4,   # Sparse regularization
    optimizer_fn=torch.optim.Adam,  
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},  
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=1,
    seed=42
)

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_val, y_val)],
    eval_name=["val"],
    eval_metric=["accuracy"],
    max_epochs=100,
    patience=10,         # Early stopping
    batch_size=1024, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

/home/ehe5bn/.local/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.97765 | val_accuracy: 0.50206 |  0:00:02s
epoch 1  | loss: 0.76887 | val_accuracy: 0.50124 |  0:00:02s
epoch 2  | loss: 0.76123 | val_accuracy: 0.49629 |  0:00:03s
epoch 3  | loss: 0.71792 | val_accuracy: 0.5136  |  0:00:03s
epoch 4  | loss: 0.69202 | val_accuracy: 0.51525 |  0:00:03s
epoch 5  | loss: 0.6731  | val_accuracy: 0.53998 |  0:00:03s
epoch 6  | loss: 0.6682  | val_accuracy: 0.50866 |  0:00:04s
epoch 7  | loss: 0.65991 | val_accuracy: 0.5169  |  0:00:04s
epoch 8  | loss: 0.66567 | val_accuracy: 0.52514 |  0:00:04s
epoch 9  | loss: 0.6622  | val_accuracy: 0.51443 |  0:00:04s
epoch 10 | loss: 0.65156 | val_accuracy: 0.50866 |  0:00:04s
epoch 11 | loss: 0.64613 | val_accuracy: 0.52844 |  0:00:05s
epoch 12 | loss: 0.64526 | val_accuracy: 0.57296 |  0:00:05s
epoch 13 | loss: 0.63995 | val_accuracy: 0.60594 |  0:00:05s
epoch 14 | loss: 0.63983 | val_accuracy: 0.59769 |  0:00:05s
epoch 15 | loss: 0.6489  | val_accuracy: 0.60511 |  0:00:05s
epoch 16 | loss: 0.6384 

/home/ehe5bn/.local/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [19]:
# Predict test
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

Accuracy: 0.5755079626578803
ROC AUC: 0.6138837105705954
              precision    recall  f1-score   support

           0       0.58      0.52      0.55      1801
           1       0.57      0.63      0.60      1841

    accuracy                           0.58      3642
   macro avg       0.58      0.57      0.57      3642
weighted avg       0.58      0.58      0.57      3642

